<a href="https://colab.research.google.com/github/Quinta10/SeaBass_parsing/blob/main/Seabass.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Я вот тут думаю.
начинать надо от данных

1. сначала найти данные

2. положение заполнить из заголовка

3. а приналичии данных в fields - заменить от туда

This specific notebook works with EXPORTS data from Seabass

The data is found in https://seabass.gsfc.nasa.gov/search#bio

We are interested in carbon data and proxy:

Chl -         Chlorophyll with its variants chla, chlb, chlc, totchl etc

POC -         Particulate organic carbon and it's options
              Note: POC_se - standard error of POC
                    POC_sd - standard deviation of POC

phyto_carbon - Carbon concentration in algae only (in vivo?)

PIC  -        Particulate inorganic carbon

PC   -        Particuate carbon (theoretically it is a sum of PIC and POC)

_se and _sd might be in other parameters too

In [25]:
#here we connect to the google drive and load libraries

from google.colab import drive

import os
import pandas as pd
import numpy as np
import re
import sys
from pathlib import Path

drive.mount('/content/drive')


SBpath = '/content/drive/MyDrive/Progress/Climate/Seabass/'
os.chdir(SBpath)
sys.path

from SB_support_changed import readSB 

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Storing the path of the data file from the Google drive
path = '/content/drive/MyDrive/Progress/Climate/Seabass/Data_11292022/'

# The data is provided as a set of tar files so we need to extract the files at least once

import tarfile
for i in range(11,12):
  fl_in = '/content/drive/MyDrive/Progress/Climate/Seabass/Data_11292022/requested_files_'+str(i)+'.tar'
  my_tar = tarfile.open(fl_in, 'r')
  dir_out = './'+str(i) #/Data_11292022
  my_tar.extractall(dir_out) # specify which folder to extract to
  my_tar.close()

# Dec1. first folder [0] gives an error
# folders 9, 10,11 are empty? If it hapens just rerun the empty folders.


# the LATEST CODE to pull DATA to a dataframe

In [77]:
# more data and some exceptions

#  this step adds date to the DataFrame from the 'fields' 
''' we can choose parameters form the list. eventually there will be regular expressions in place to find all parameters that have part of the word'''
# we keep the previous cell in case i mess up this one and don't remember how to go back
# focus_param = ['poc','pic','pc','phyto_carbon',
#               'chl','chla','tchl','tot_chl_a']

# focus_params  - a list of parameters we are interested right now
# focus_param = ['chl','chla','tchl','tot_chl_a']
focus_param = ['phyto_carbon']
# coordinates
loc_param = ['date','time','time zone',
            'depth','pressure',
            'lon','lat','station','file_name']
# some parameters are obtained with different methods. we save the method to understand possible discrepancies
# param_methods = ['poc_m','pic_m','pc_m','phyto_c_m',
#               'chl_m','chla_m','tchl_m','tot_chl_a_m']
param_methods = ['phyto_carbon']

# the table we want to get at the end (should be formed automatically)
# full_list_params = ['date','time','time zone',
#             'depth','pressure',
#             'lon','lat','station','file_name',
#             'poc','pic','pc','phyto_carbon',
#               'chl','chla','tchl','tot_chl_a']
full_list_params = ['date','time','time zone',
            'depth','pressure',
            'lon','lat','station',
            'phyto_carbon','c_units','file_name']

faulty_list = ['prr_car56_1_2.sb','2013-043-172854.sb','DEL1004_Station_048_CTD_Data.sb'] #,'f19961108175753.P02.sb','f19961025141420.R01.sb','f19961113151047.R01.sb'


param_short_list = set(focus_param)
param_long_list = set(full_list_params)
chk=1
def SB_files_fields(P_in, param_short_list,param_long_list): # the name of the path here was messing this function
  for root, dirs, files in os.walk(P_in, topdown=False): 
      for name in files:
        for i in range(0, len(faulty_list)):              # STEP 2
          if name == faulty_list[i]:
            name = 'faulty'

        if name.endswith('.sb'):
          p_s = set(readSB(os.path.join(root, name),no_warn=True).headers['fields'].lower().split(','))
          common_short_param = param_short_list.intersection(p_s)
          if common_short_param:  # 
            file_loc = os.path.join(root, name)
            common_long_param = param_long_list.intersection(p_s)
            # print(common_long_param)  # 
            df_data = pd.DataFrame(readSB(file_loc,no_warn=True).data)
            res_df = df_data[common_long_param] 
            # time zone 
            time_z = readSB(file_loc,no_warn=True).headers['start_time'][9:12]
            res_df = res_df.assign(time_zone=time_z) # hours/minutes ets
            # file name
            full_f_n = os.path.join(root, name)
            res_df = res_df.assign(file_name=full_f_n[40:])
            # need to write a different function for this?
            if 'date' not in common_long_param:
              # how to write a df column? 
              date_in = readSB(file_loc,no_warn=True).headers['start_date']
              # res_df['date'] = date_in # works with a warning
              res_df = res_df.assign(date=date_in)
            if 'time' not in common_long_param:
              time_in = readSB(file_loc,no_warn=True).headers['start_time'][0:8]
              res_df = res_df.assign(time=time_in) # hours/minutes ets
            if 'lat' not in common_long_param:
              llat_in = readSB(file_loc,no_warn=True).headers['north_latitude']
              lat_in = re.split(r'\[', llat_in)[0]
              res_df = res_df.assign(lat=lat_in) # hours/minutes ets
            if 'lon' not in common_long_param:
              llon_in = readSB(file_loc,no_warn=True).headers['east_longitude']
              lon_in = re.split(r'\[', llon_in)[0]
              res_df = res_df.assign(lon=lon_in) # hours/minutes ets
            # -------------------------------
            #  NEED TO ADD UNITS. this solution is for 1 parameter only
            #--------------------------------
            p_list =  list(p_s)
            # print(p_list)
            if focus_param in p_list:
              index = p_list.index(focus_param)
              unit_index  =   (readSB(os.path.join(root, name),no_warn=True).headers['units'].lower().split(','))[index]
              res_df = res_df.assign(units = unit_index) 
            #  end of a temporary solution
            #----------------------------------
            yield res_df, common_long_param  # 


        # if name == 'prr_car56_1_2.sb':
        #   name = 'faulty' 
        # elif name =='f19961108175753.P02.sb':
        #   name = 'faulty file either missing data or wrong formating'
        # elif name == 'f19961025141420.R01.sb'
        #   name = 'faulty'
        

In [ ]:
# by changing the path we can switch between the whole database
# to specific experiemnts or data holding institutions
# for i in range(7,11):
nums = [2,3,7,8,9,10,11]
for i in nums:
  P_in = "/content/drive/MyDrive/Progress/Climate/Seabass/SB_11292022/"+str(i)
  # P_in = "/content/drive/MyDrive/Progress/Climate/Seabass/EXPORTS/MBARI/"
  # fl_list = 0
  # create an Empty DataFrame object
  df_big = pd.DataFrame()

  for f_n, C in SB_files_fields(P_in,param_short_list,param_long_list ):
   df_big = df_big.append(f_n, ignore_index = True) 
  


  # df_big
  # this works. saving here coz it's running at night
filepath = Path('/content/drive/MyDrive/Progress/Climate/Seabass/Carbon_Night_c_phyto3.cvs')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
df_big.to_csv(filepath)
  

In [74]:
df_big.tail()
# 5823 x 3

""


In [55]:
# from pathlib import Path 
# compression_opts = dict(method='zip',
#                         archive_name='Carbon_EXPORTS.zip')   
filepath = Path('/content/drive/MyDrive/Progress/Climate/Seabass/Carbon_EXPORTS_c_phyto3.cvs')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
df_big.to_csv(filepath) 

In [ ]:
# do we need to zip? 

**Sand box**

In [3]:
#  this step adds date to the DataFrame from the 'fields' 
''' we can choose parameters form the list. eventually there will be regular expressions in place to find all parameters that have part of the word'''
# we keep the previous cell in case i mess up this one and don't remember how to go back
# focus_param = ['poc','pic','pc','phyto_carbon',
#               'chl','chla','tchl','tot_chl_a']

# focus_params  - a list of parameters we are interested right now
focus_param = ['chl','chla','tchl','tot_chl_a']
# coordinates
loc_param = ['date','time',
            'depth','pressure',
            'lon','lat','station']
# some parameters are obtained with different methods. we save the method to understand possible discrepancies
param_methods = ['poc_m','pic_m','pc_m','phyto_c_m',
              'chl_m','chla_m','tchl_m','tot_chl_a_m']

# the table we want to get at the end (should be formed automatically)
full_list_params = ['date','time',
            'depth','pressure',
            'lon','lat','station',
            'poc','pic','pc','phyto_carbon',
              'chl','chla','tchl','tot_chl_a']


param_short_list = set(focus_param)
param_long_list = set(full_list_params)

def SB_files_fields(P_in, param_short_list,param_long_list): # the name of the path here was messing this function
  for root, dirs, files in os.walk(P_in, topdown=False): 
      for name in files:
        if name == 'prr_car56_1_2.sb':
          continue 
        if name.endswith('.sb'):
          p_s = set(readSB(os.path.join(root, name),no_warn=True).headers['fields'].lower().split(','))
          common_short_param = param_short_list.intersection(p_s)
          if common_short_param:  # 
            file_loc = os.path.join(root, name)
            common_long_param = param_long_list.intersection(p_s)
            # print(common_long_param)  # 
            df_data = pd.DataFrame(readSB(file_loc,no_warn=True).data)
            res_df = df_data[common_long_param] 
            # need to write a different function for this?
            if 'date' not in common_long_param:
              # how to write a df column? 
              date_in = readSB(file_loc,no_warn=True).headers['start_date']
              # res_df['date'] = date_in # works with a warning
              res_df = res_df.assign(date=date_in)
            if 'time' not in common_long_param:
              time_in = readSB(file_loc,no_warn=True).headers['start_time'][0:8]
              res_df = res_df.assign(time=time_in) # hours/minutes ets
            if 'lat' not in common_long_param:
              llat_in = readSB(file_loc,no_warn=True).headers['north_latitude']
              lat_in = re.split(r'\[', llat_in)[0]
              res_df = res_df.assign(lat=lat_in) # hours/minutes ets
            if 'lon' not in common_long_param:
              llon_in = readSB(file_loc,no_warn=True).headers['east_longitude']
              lon_in = re.split(r'\[', llon_in)[0]
              res_df = res_df.assign(lon=lon_in) # hours/minutes ets


            yield res_df, common_long_param  # 




По поводу того, что нужно будет сделать: 
Нужно решить проблему с тем, что в таблице могут быть разные комбинации параметров, которые относятся к дате/времени/координатам
Для этого: 
1) Нужно написать функцию, которая будет говорить, есть ли указанный параметр в данных или нет 

Done


2) На основе этой функции фильтровать исходный набор параметров, относящихся к расположению

Done

3) После фильтрации для параметров, которые есть в данных, взять значения из таблицы, для остальных взять значения из заголовка 

in progress

Ещё можно подумать над тем, как изменить данные, которые мы берём из заголовков, они немного немного не в том виде

regEx for the deg, 

In [ ]:
# put the above cell in a loop?
l_p = set(loc_param)

for i in range(2):   # how many files again? How do we know? Do we need to know? 6101
     FL = next(SB_files_fields(P_in)).lower().split(',')
     # expectation: sets intersection() method will give us the list of common words
     fl_set = set(FL)
     C = l_p.intersection(fl_set)
     print(C)
# we see the date in the fields but in the set there is no info
# now it works: the reason was : the function returned a string. we had to split it.

date,sample,bottle,lat,lon,pressure,sigma_theta,sal,wt,oxygen_kg,ratio_abs578_abs434,pH,pH_1id,pH_quality
{'date', 'pressure', 'lat', 'lon'}
date,sample,bottle,lat,lon,pressure,sigma_theta,sal,wt,oxygen_kg,ratio_abs578_abs434,pH,pH_1id,pH_quality
{'date', 'pressure', 'lat', 'lon'}


In [ ]:
# How do we know the set intersection is not sufficient?
# maybe we should no worry about it? But read all the header has?
# and the match the two sets again. 
# mind that pressure for us is as good as depth. 

# i think i need a similar function to read the header

In [ ]:
def SB_files_header(SB_path_in): # the name of the path here was messing this function
  for root, dirs, files in os.walk(SB_path_in, topdown=False): 
      for name in files:
        if name.endswith('.sb'):
          header_set = readSB(os.path.join(root, name),no_warn=True).headers
          yield header_set
# I am using the same function that reads fields (as 
#fields are in the same header)

In [ ]:
# playing with regex
import re
long_in = read_head['west_longitude']
result = re.split(r'\[', long_in)
print(result[0])

-145.7563


In [ ]:
read_head = next(SB_files_header(P_in))
# problem 1 not every header has all the keys
# problem 2 - get rid of the text part (reg expressions?)
# print(read_head)

print(read_head['start_date'])
print(read_head['end_date'])
print(read_head['start_time'][0:8]) # just cut GMT off
print(read_head['end_time'][0:8])
# print(read_head['station']) # not everywhere
# i think there is no way we do not get these values. the files cannot be without these
print(re.split(r'\[', read_head['west_longitude'])[0])
print(re.split(r'\[', read_head['east_longitude'])[0])
print(re.split(r'\[', read_head['north_latitude'])[0])
print(re.split(r'\[', read_head['south_latitude'])[0])




# ''' we need: keys: data_file_name
#            start_date,
#            end_date,
#            start_time, 19:27:00[GMT]
#            'water_depth'
#            end_time,
#            'station',
#            'west_longitude', '-145.7563[DEG]',
#            'east_longitude', '-144.4505[DEG]',
#            'north_latitude', '51.042[DEG]', 
#            'south_latitude', '49.948[DEG]'
#     one of the issues is that the coord are with [DEG]
#     and the time is with [GMT]  ---> Q: is it always like that?
#     So we pull these data out same way as we pull fields.
#     next step we fill
#           '''

20180814
20180909
00:00:00
23:59:59
-145.7563
-144.4505
51.042
49.948


In [ ]:
# l_p = set(loc_param)
# print(type(l_p))
# FL = set(next(SB_files_fields(P_in)).lower().split(','))
# print(len(FL))

In [ ]:
# this is a test. I don't understand, why my function is stuck either at the same value or gives "stopItteration"
# possible reasons: i stop itteratio by somneerting the output  to list or using a print function. 
# this simple itterator with a condition, works
import random
def try_gen(a,b):
    r = random.randint(1, 8)
    
    if r <5 :
      bb = r
    else: bb = 20
    yield bb,r

In [ ]:
some_list = []
for i in range(5):
    # for ii in try_gen(1,8):
    s_l = next(try_gen(1,8))
    some_list.append(s_l[1])

print(some_list)
# this works

[6, 1, 5, 7, 2]


In [ ]:
# let's compare it to the parameter list that I am interested in
# parameters we want
focus_param = ['poc','pic','pc','phyto_c',
              'chl','chla','tchl','tot_chl_a']
# coordinates
loc_param = ['date','time',
            'depth','pressure',
            'lon','lat','station']
param_methods = ['poc_m','pic_m','pc_m','phyto_c_m',
              'chl_m','chla_m','tchl_m','tot_chl_a_m']
#  exceptions: 
# poc_sd, poc_se, prochlorococcus, picoeukaryote, Bchl_a

In [ ]:
# if the set of the generator coinsides with carb_param - we create a dataframe and write the data
res_df = pd.DataFrame(columns = loc_param+
                      focus_param+
                      param_methods)
# res_df


In [ ]:
f = SB_files_fields(sbPath)
for ii in range(101):  # there are 6101 files
    fields_from_file = next(f).split(',')
# I need to return the actual overlaps not just yes/no. 
    # print(fields_from_file)
    check_in = (set(focus_param) & set(fields_from_file))
    # print(check_in)
    if len(check_in)>0:
      print(check_in)

In [ ]:


#  first take on the parameters. It might happen that chl will have to be extended to more than 1 column
# chl, poc, phyto_carbon, pic, pc
# paramters in the files are written with upper and lower case randomly
param_set = {'chl', 'poc', 'phyto_carbon', 'pic', 'pc'}



In [ ]:
def SB_files_fields(SB_path_in): # the name of the path here was messing this function
  for root, dirs, files in os.walk(SBpath, topdown=False): 
      for name in files:
        if name.endswith('.sb'):
          # print(os.path.join(root, name))
          param_set = readSB(os.path.join(root, name),no_warn=True).headers['fields']
          #      read the file           'fields' only from the header, turn to lower case cannot assign if slpit
          print(param_set)
          yield param_set

          # what if the file doesn't read? Coding is wrong? Coding is fixed with the new support file


if there is no "date" in this string (start_date  from  .headers) maybe it makes sense to take it from headers from the start

if there is no depth, but pressure (make a note "method"?)

if date in year mon day

if time in hour minute second

if lon in logitude

if lat in latitude

In [ ]:
# testing
ii = 0
for root, dirs, files in os.walk("/gdrive/MyDrive/Progress/Climate/Seabass/EXPORTS/", topdown=False): 
  for name in files:
        if name.endswith('.sb'):
          ii+=1
print(ii)

6101


**TAR files excersises**

In [ ]:
import tarfile,os
import sys
os.chdir("/content/drive/MyDrive/Progress/Climate/Seabass/Data_11292022")
tar = tarfile.open("requested_files_2.tar")
for member in tar.getmembers():
    f=tar.extractfile(member)
    content=f.read()
    print ("%s has %d newlines" %(member, content.count("\n")))
    print ("%s has %d spaces" % (member,content.count(" ")))
    print ("%s has %d characters" % (member, len(content)))
    sys.exit()
tar.close()

In [ ]:
arch_file_lib = []
with tarfile.open('/content/drive/MyDrive/Progress/Climate/Seabass/Data_11292022/requested_files_2.tar','r') as archive:
  arch_file_lib.append(archive.list())

Выходные данные были обрезаны до нескольких последних строк (5000).
?rw-rw-r-- sdps/sdps      15657 2022-11-29 03:08:03 requested_files/UCSB/CRSEO/PnB/pb/archive/ctd/X080422E.sb 
?rw-rw-r-- sdps/sdps     188472 2022-11-29 03:10:58 requested_files/UCSB/CRSEO/PnB/pb/archive/ctd/X150618G.sb 
?rw-rw-r-- sdps/sdps      21497 2022-11-29 03:07:50 requested_files/UCSB/CRSEO/PnB/pb/archive/ctd/X101216E.sb 
?rw-rw-r-- sdps/sdps     329741 2022-11-29 03:08:11 requested_files/UCSB/CRSEO/PnB/pb/archive/ctd/X130605A.sb 
?rw-rw-r-- sdps/sdps    1545776 2022-11-29 03:10:05 requested_files/UCSB/CRSEO/PnB/pb/archive/ctd/X151015X.sb 
?rw-rw-r-- sdps/sdps       9142 2022-11-29 03:07:44 requested_files/UCSB/CRSEO/PnB/pb/archive/ctd/X100226A.sb 
?rw-rw-r-- sdps/sdps     637080 2022-11-29 03:07:50 requested_files/UCSB/CRSEO/PnB/pb/archive/ctd/X111117B.sb 
?rw-rw-r-- sdps/sdps       5408 2022-11-29 03:06:57 requested_files/UCSB/CRSEO/PnB/pb/archive/ctd/X061212A.sb 
?rw-rw-r-- sdps/sdps    1119361 2022-11-29 0